## PLST - Phuture Liquid Staking Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0
min_weight = 0.005
max_weight = 0.15
circ_supply_threshold = 0
liveness_threshold = 30 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["liquid-staking-tokens","liquid-restaking-tokens"]

# Calculating max slippage based on liquidity threshold
liq = 1.5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'arbitrum-one',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    remove_category_assets=['frax-ether'],
    values_to_update=[{'id':'benqi-liquid-staked-avax','category':'symbol','value':"sAVAX"},
                      {'id':'rocket-pool-eth','category':'symbol','value':"rETH"},
                      {'id':'staked-ether','category':'symbol','value':"stETH"},
                      {'id':'staked-frax-ether','category':'symbol','value':"sfrxETH"},
                      {'id':'stader-ethx','category':'symbol','value':"ETHx"},
                      {'id':'stakewise-v3-oseth','category':'symbol','value':"osETH"}
                      ],
    onchain_oracles=['coinbase-wrapped-staked-eth','lido-staked-matic','sweth','kelp-dao-restaked-eth','stader-maticx','wrapped-eeth']
    )
liquidity

/Users/olivermehr/Documents/GitHub/rebalancing-data/PLST/../methodology.py:136: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.category_data = pd.concat(dataframe_list, axis=0)


Excluding dinero-staked-eth, pricing data available only for 1 < 30 days
mantle not supported
sweth on arbitrum-one returned an invalid API response
Manually calculating slippage for staked-frax-ether on ethereum
moonbeam not supported
staked-frax-ether on fantom returned an invalid API response
Manually calculating slippage for staked-frax-ether on optimistic-ethereum
Manually calculating slippage for staked-frax-ether on arbitrum-one
staked-frax-ether on polygon-pos returned an invalid API response
staked-frax-ether on binance-smart-chain returned an invalid API response
stader-maticx on ethereum returned an invalid API response
ankreth on fantom returned an invalid API response
Manually calculating slippage for ankreth on optimistic-ethereum
mode not supported
ankreth on avalanche returned an invalid API response
polygon-zkevm not supported
trufin-staked-matic on ethereum returned an invalid API response
Manually calculating slippage for trufin-staked-matic on polygon-pos
bridge-ora

,arbitrum-one,avalanche,base,binance-smart-chain,ethereum,fantom,optimistic-ethereum,polygon-pos,optimal chain,best slippage,best slippage chain
id,,,,,,,,,,,
wrapped-eeth,-0.002653,NaN,NaN,NaN,-0.001193,NaN,NaN,NaN,arbitrum-one,-0.001193,ethereum
staked-frax-ether,-0.001524,NaN,NaN,NaN,-0.097773,NaN,-0.008001,NaN,arbitrum-one,-0.001524,arbitrum-one
stakewise-v3-oseth,NaN,NaN,NaN,NaN,-0.001680,NaN,NaN,NaN,ethereum,-0.001680,ethereum
rocket-pool-eth,-0.001755,NaN,-0.004276,NaN,-0.003089,NaN,-0.006962,-0.976813,arbitrum-one,-0.001755,arbitrum-one
staked-ether,NaN,NaN,NaN,NaN,-0.002010,NaN,NaN,NaN,ethereum,-0.002010,ethereum
ankreth,-0.002147,NaN,NaN,-0.502575,-0.002323,NaN,-0.008292,NaN,arbitrum-one,-0.002147,arbitrum-one
mantle-staked-ether,NaN,NaN,NaN,NaN,-0.003261,NaN,NaN,NaN,ethereum,-0.003261,ethereum
coinbase-wrapped-staked-eth,-0.005772,NaN,-0.006711,NaN,-0.003309,NaN,-0.996492,-0.780565,arbitrum-one,-0.003309,ethereum
kelp-dao-restaked-eth,NaN,NaN,NaN,NaN,-0.003315,NaN,NaN,NaN,ethereum,-0.003315,ethereum


In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
staked-ether,stETH,3.447507e+10,0.150000,9831,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,ethereum
rocket-pool-eth,rETH,2.122046e+09,0.150000,9831,0xec70dcb4a1efa46b8f2d97c310c9c4790ba5ffa8,arbitrum-one
mantle-staked-ether,METH,1.895703e+09,0.150000,9831,0xd5f7838f5c461feff7fe49ea5ebaf7728bb0adfa,ethereum
wrapped-eeth,WEETH,1.391478e+09,0.150000,9831,0x35751007a407ca6feffe80b3cb397736d2cf4dbe,arbitrum-one
sweth,SWETH,8.092988e+08,0.090846,5953,0xf951e335afb289353dc249e82926178eac7ded78,ethereum
staked-frax-ether,sfrxETH,7.146457e+08,0.080221,5257,0x95ab45875cffdba1e5f451b950bc2e42c0053f39,arbitrum-one
kelp-dao-restaked-eth,RSETH,6.752365e+08,0.075797,4967,0xa1290d69c65a6fe4df752f95823fae25cb99e5a7,ethereum
coinbase-wrapped-staked-eth,CBETH,6.751044e+08,0.075782,4966,0x1debd73e752beaf79865fd6446b0c970eae7732f,arbitrum-one
benqi-liquid-staked-avax,sAVAX,4.033769e+08,0.045280,2967,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,avalanche
